In [1]:
# CARREGANDO BIBLIOTECAS
import pymongo
from pymongo import MongoClient
import pandas as pd
import os

import networkx as nx

import pickle
import gc
from tqdm import tqdm_notebook as tqdm

path_atual = os.getcwd()
print(path_atual)
path_download = '../../Downloads/'

C:\Users\Walter\Documents\WOS_CitationNetworks


In [2]:
category = 'LOGIC'

## 1. Importando os dados. Criando os banco de dados.

In [3]:
# CONEXÃO
client = MongoClient('mongodb://localhost:27017')

In [4]:
# SELECIONANDO UM BANCO
db = client.CitationsNetwork_LOGIC
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'CitationsNetwork_LOGIC')

In [5]:
# ARTIGOS E PERIÓDICOS DE ORIGEM
with open('pickles/' + category + '_INFOS.pickle', 'rb') as f:
    [info_articles, info_journals] = pickle.load(f)

In [6]:
source_articles = info_articles.keys()
print('Total de Artigos publicados na subárea Lógica da Matemática: ' + str(len(source_articles)) + '.')
source_journals = info_journals.keys()
print('Total de Periódicos publicáveis na subárea Lógica da Matemática: ' + str(len(source_journals)) + '.')

Total de Artigos publicados na subárea Lógica da Matemática: 18718.
Total de Periódicos publicáveis na subárea Lógica da Matemática: 18.


In [7]:
old_keys = info_articles

Coleções estão para o mongodb como tabelas estão para bancos relacionais como postgres ou mysql. Collections são grupos de documentos armazenados no mongodb, é nelas onde os dados serão armazenados, a selecão de uma collection é feita da seguinte forma:

In [8]:
# CRIANDO COLEÇÕES
articles = db.articles
journals = db.journals
citations = db.citations

Dados no mongodb são representados e armazenados usando o estilo de documentos JSON. Note que esses documentos podem conter listas, além de tipos nativos do python, no caso uma instância de datetime.datetime.

Para inserir um documento em uma collection usamos o método `insert_one()`. Quando um documento é inserido, um “ObjectId” é criado, se você não especificar um “_id” em seu documento, será atribuido um valor aleatório ao seu ObjectId, veja no exemplo:

In [9]:
# TODOS OS ARTIGOS E PERIÓDICOS (INCLUSIVE AS REFERÊNCIAS)
with open('pickles/' + category + '_REFS.pickle', 'rb') as f:
    [info_articles, info_journals, ref_error] = pickle.load(f)
    #[info_articles, info_journals] = pickle.load(f)

In [10]:
all_articles = info_articles.keys()
print('Total de Artigos da rede: ' + str(len(all_articles)) + '.')
all_journals = info_journals.keys()
print('Total de Periódicos da rede: ' + str(len(all_journals)) + '.')

Total de Artigos da rede: 187938.
Total de Periódicos da rede: 52267.


In [11]:
# Caso esteja na vesrsão antiga dos pickles
#list(source_articles)[0] -> '2-s2.0-58049151597'
#source_articles[0] - > '58049151597'
def convertIdArticles(source_articles, info_articles, old_keys):
    
    source_articles_aux = []
    for art in list(source_articles):
        source_articles_aux.append(art.split('2-s2.0-')[1])
        
    for art in source_articles_aux:
        try: 
            del info_articles['2-s2.0-' + art]
            del info_articles[art]
        except:
            continue
            
    new_keys = {}
    for art in source_articles_aux:
        new_keys[art] = old_keys['2-s2.0-' + art]
    new_keys.update(info_articles)

    return source_articles_aux, new_keys

In [12]:
source_articles, info_articles = convertIdArticles(source_articles, info_articles, old_keys)

In [13]:
all_articles = info_articles.keys()
print('Total de Artigos da rede: ' + str(len(all_articles)) + '.')
all_journals = info_journals.keys()
print('Total de Periódicos da rede: ' + str(len(all_journals)) + '.')

Total de Artigos da rede: 179054.
Total de Periódicos da rede: 52267.


In [14]:
# INSERINDO UM DOCUMENTO NO MONGO
for journal in all_journals:
    id_j = journals.insert_one(info_journals[journal]).inserted_id

In [15]:
for article in all_articles:
    id_art = articles.insert_one(info_articles[article]).inserted_id

## 2. Montando a Rede de Citações.

Criando índices de consulta no MongoDB.

In [17]:
articles.create_index([('ID_A',pymongo.ASCENDING)])

'ID_A_1'

In [18]:
def citationsNetwork(source_articles, all_articles, info_articles):
    
    info_citations = {}
    e = []
    n = len(source_articles)
    
    for i in tqdm(range(n)):
        art = i + 1
        infos = articles.find_one({'ID_A': art})
        id_a = infos['ID_A']
        id_j = infos['ID_J']
        year = int(infos['YEAR'].split('-')[0])
        refs = infos['REFS']
        
        for ref in refs:
            if info_articles.get(ref) == None: 
                e.append([art,ref])
                #print('>>> ERRO na referência ' + str(ref) + ' do artigo ' + str(art) + '. <<<')
                continue
            
            info_ref = info_articles[ref]
            cit = {'SOURCE_A_ID': id_a, 
                   'SOURCE_A_YEAR': year,
                   'SOURCE_J_ID': id_j,
                   'TARGET_A_ID': info_ref['ID_A'],
                   'TARGET_A_YEAR': int(info_ref['YEAR'].split('-')[0]),
                   'TARGET_J_ID': info_ref['ID_J'],
                   'EDITIONS': int(info_ref['YEAR'].split('-')[1])
                  }
            info_citations.update(cit)
            id_cit = citations.insert_one(cit).inserted_id
        #print('Citações do Artigo ' + str(art) + ' armazenadas!')
        gc.collect()
        pass
    return info_citations, e

In [19]:
info_citations, e = citationsNetwork(source_articles, all_articles, info_articles)

A Jupyter Widget

In [20]:
len(e)

27

In [21]:
var_cit = pd.DataFrame(list(citations.find()), columns=['SOURCE_A_ID', 'SOURCE_A_YEAR', 'SOURCE_J_ID', 'TARGET_A_ID' , 'TARGET_A_YEAR' ,'TARGET_J_ID' ,'EDITIONS'])

In [22]:
var_cit.index+=1

In [23]:
var_cit.head()

,SOURCE_A_ID,SOURCE_A_YEAR,SOURCE_J_ID,TARGET_A_ID,TARGET_A_YEAR,TARGET_J_ID,EDITIONS
1,1,2008,1,1232,2003,1,12
2,1,2008,1,18720,2003,19,1
3,1,2008,1,18102,2005,1,9
4,1,2008,1,9849,1973,1,7
5,1,2008,1,18723,1993,20,1


In [24]:
var_cit.tail()

,SOURCE_A_ID,SOURCE_A_YEAR,SOURCE_J_ID,TARGET_A_ID,TARGET_A_YEAR,TARGET_J_ID,EDITIONS
326860,18718,2015,6,24749,2011,1573,5
326861,18718,2015,6,187937,0,52267,8
326862,18718,2015,6,178757,2010,109,12
326863,18718,2015,6,17447,2011,7,4
326864,18718,2015,6,23186,1996,108,10


In [25]:
#var_cit.to_csv('outputs/var_cit_' + category + '.csv', index=False)

In [26]:
# gephi = var_cit = pd.DataFrame(list(citations.find()), columns=['SOURCE_J_ID', 'TARGET_J_ID'])
# gephi.index+=1
# gephi = gephi.rename(columns={'SOURCE_J_ID': 'SOURCE', 'TARGET_J_ID': 'TARGET'})

In [27]:
#gephi.to_csv('outputs/gephi_' + category + '.csv', index=False)